In [ ]:
#  ------------------------------------------------------------------------------------------
# Libraries
#  ------------------------------------------------------------------------------------------
from tkinter import *
from tkinter.ttk import *
import time, requests, pandas as pd, re
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings; warnings.simplefilter('ignore')